<a href="https://colab.research.google.com/github/miluseel/MiluseelAI/blob/main/Chap5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 5-2. 교차 검증과 그리드 서치

### 검증세트
: 모델을 튜닝하고 과적합을 막기 위한 증간 점검용 데이터

* 보통 2-30%를 테스트세트와 검증세트로 떼어놓음
* 훈련 세트에서 모델훈련 하고 검증세트로 모델 평가 <br>-> 태스트하고 싶은 매개변수 바꿔가며 가장 좋은 모델 고름 <br>-> 매개변수를 사용해 훈련/검증세트 합쳐 전체 훈련세트로 모델 다시 훈련 <br>-> 테스트세트에서 최종 점수 평가 <br>
: 실전 투입 시 테스트세트의 점수와 비슷한 성능 기대

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
#class열을 타깃으로 사용 / 나머지는 특성
data = wine[['alcohol','sugar','pH']]
target = wine['class']

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
#훈련세트, 검증세트만들기
#매개변수 test_size를 0.2로 지정
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
#train_test_split() 2 번적용한 훈련/검증 세트 크기 확인
print(sub_input.shape, val_input.shape)

#-> 훈련세트 : 4157 , 검증세트 : 1040

(4157, 3) (1040, 3)


In [ ]:
#훈련/검증세트의 input, target을 사용해 모델 생성 -> 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

#과적합 되었음을 확인

0.9971133028626413
0.864423076923077


### 교차검증
: 검증세트 세트를 떼어 내어 평가하는 과정을 여러 번 반복 -> 이 점수를 평균하여 최종 검증 점수

- k-폴드 교차검증 (k-fold cross validation) : 훈련 세트를 k부분으로 나눠서 교차검증을 수행하는 것<br>
  - sklearn : cross_validate()

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01030755, 0.00995803, 0.01016951, 0.01012635, 0.0097363 ]), 'score_time': array([0.0024879 , 0.00248218, 0.00235271, 0.00241494, 0.00227833]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


* cross_validate()은 훈련세트를 섞어 폴드에 나누지 않음
 - 교차 검증 시 훈련세트를 섞으려면 분할기 지정
  - 희귀모델일 경우 : KFold분할기 사용
  - 분류모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 사용

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#훈련세트 섞은 후 10폴드 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


### 하이퍼 파라미터 튜닝 <br>
: 클래스나 메서드의 매개변수로 표현 <br>

* 하이퍼 파라미터 튜닝작업 <br>
: 라이브러리가 제공하는 기본값 사용  <br>-> 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꿈 <br>
: 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술 <br> - AutoML
: max_depth의 최적값은 min_samples_split매개변수의 값이 바뀌면 함께 달라짐 <br> _ 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 함

* Grid Search - sklearn <br> : 하이퍼파리미터 탐색과 교차 검증을 한 번에 수행하여 cross_validate()호출 필요 없음

In [ ]:
#GridSearchCV 임포트 - 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦
# 0.0001씩 증가하는 5개의 값을 시도

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
#GridSearchCV클래스에 탐색 대상 모델과 params변수 전달
# 그리드 서치 객체 형성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

* fit호출 - min_impurity_decrease바꿔가며 결정트리 모델 사용 -> 5번 교차검증
* cv 매개변수 기본값 5 ( 5 * 5  모델 훈련 )
* GridSearchCV 클래스의 n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수 지정
* 기본값 1 / -1 지정시 시스템의 모든 코어 사용가능

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

* 사이킷런 그리드 서치는 훈련이 끝나면 교차 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델 훈련
* gs객체의 best_estimator_ 속성에 저장 (+)

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# 서치로 찾은 최적의 매개변수 : 0.0001
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# cv_results_ 의 mean _test_score : 각 매개 변수에서 수행한 교차검증의 평균점수 ( 5번 교차 검증 후 )
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


* gs 객체의 best_index_속성 사용하여 가장 높은 값의 인덱스 추출
* 이 인덱스를 이용해 params키에 저장된 매개변수 출력<br>
=> 최상의 검증 점수를 만든 매개변수의 조합

In [ ]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수 지정
2. 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음 -> 그리드 서치 객체에 저장
3. 최상의 매개변수에서 전체 훈련세트를 사용해 최종모델 훈련 -> 그리드 서치 객체에 저장

* min_impurity_decrease : 노드를 분할가이 위한 분순도 감소 최소량 지정
* max_depth : 트리의 깊이 제한
* min_samples_split : 노드를 나누기 위한 최소한 샘플 수

In [ ]:
import numpy as np

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)
          }

* numpy arange() : 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열 생성
-> 0.0001 에서 시작하여 0.001이 될 때까지  0.0001을 계속 더한 배열<br>
-> 두 번째 매개변수는 포함되지 않으므로 배열의 원소는 총 9개

* 파이썬 range() : 정수만 사용가능 <br>
- max_depth를 5 - 20 까지 1씩 증가하면서 15개의 값을 생성
- min_samples_split 2-100까지 10씩 증가하면서 10개의 값 생성

* 교차 검증 횟수 : 9 * 15* 10 = 1350
- 기본 5-fold교차 검증수행 : * 5 = 6750


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# n_jobs 매개변수를 -1 설정 -> 그리드 서치 실행

gs=GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
#최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [ ]:
#최상의 교차검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤서치
* 매개변수의 값이 수치일 떄 값의 범위나 간격을 미리 정하기 어려울 때
* 너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래 걸릴 때
- ( 매개 변수값의 목록을 전달하는 것이 아니라 ) 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달


### 싸이파이 (scipy)
: 적분, 보간, 선형대수, 확률등을 포함한 수치 계산 전용 라이브러리

- stats
  - uniform, randint : 주어진 범위 내에서 고르게 값을 선택<br>
    : 균등 분포에서 샘플링
    - randint : 정수값 추출
    - uniform : 실수값 추출


In [ ]:
from scipy.stats import uniform, randint

In [ ]:
# 10개에서 랜덤 추출
rgen = randint(0,10)
rgen.rvs(10)

array([5, 4, 5, 7, 8, 0, 7, 3, 7, 7])

In [ ]:
# 1000개에서 랜덤 추출
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 86, 107, 121, 116, 110,  88, 110,  71,  93,  98]))

In [ ]:
# 0- 1사이의 실수값 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([8.88403216e-01, 9.20702111e-01, 6.54433590e-05, 3.60964075e-01,
       1.69659583e-01, 2.63437942e-01, 1.70295822e-01, 3.90843146e-01,
       1.96121360e-01, 6.93570575e-01])

* min_samples_leaf : 리프노드가 되기 위한 최소 샘플의 개수<br>
- 노드가 분할하여 만들어질 자식 노드의 샘플수가 이 값보다 작을 경우 분할 멈춤

In [ ]:
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25),
          }

* 샘플링 횟수 : sklearn
  - RandomizedSearchCV : n_iter지정

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs = -1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7daddd1fc0b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7dadda0a8ec0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7dadda0aad20>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7daddb7352b0>},
                   random_state=42)

In [ ]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
#최고의 교차 검증점수 확인
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


* 최적의 모델은 전체 훈련 세트 (train_input,train_target)로 훈련되어 best_estimator_속성에 저장

In [ ]:
#테스트 세트의 성능확인
df = rs.best_estimator_
print(df.score(test_input, test_target))

0.86


### < 문제해결과정 복기 >
: 최적의 모델을 위한 하이퍼파라미터 탐색
* 선별 작업의 성능을 올리기 위해 결정트리의 다양한 하이퍼 파라미터를 시도해 봐야 함
* 테스트 세트는 최종모델 선택까지 사용하지 말아야 함
* 테스트 세트를 사용하지 않고 모델평가 하려면 모델세트 필요 ( 검증세트 / 개발세트 )
* 검증세트는 훈련세트 중 일부를 다시 덜어내어 생성
* 훈련 모델의 성능을 안정적으로 평가하기 위해 검증세트를 여러번 반복 ( 교차검증 ) - 보통 5/10등분
* 폴드( 나누어진 한 덩어리 ) - 한 폴드씩 돌아가면서 검증세트 역할
* 최종 점수는 모든 폴드의 검증 점수를 평균하여 계산
* 그리드 서치 : 테스트 하고 싶은 매개변수 리스트를 만들어 이 과정을 자동화
* 싸이파이 : ( 매개 변수 값이 수치형 / 연속적인 실수값 )


### < 함수 리마인드 >
scikit-learn
- cross_validate() : 교차검증을 수행하는 함수
  - scoring매개변수: 검증에 사요할 평가 지표 지정
  - cv매개변수 : 교차검증폴드수, 분할기 객체 지정 ( 기본값 5 ) : 5-fold 교차검증
    - 회귀 : k-flod클래스
    - 분류 : stratifiedKFold 클래스
  - n-jobs : 교차 검증 수행할 떄 사용할 CPI코어 수 지정 ( 기본값 1 ) (-1 시스템에 있는 모든 코어 사용 )
  - return_train_score 매개변수 : True지정 시 훈련 세트의 점수 반환 ( 기본값: false )
- GridSearchCV : 교차검증 하이퍼 파리미터 탐색
  - 최상의 모델 찾은 후 훈련세트 전체를 사용해 최종모델 훈련
  - 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체 전달
  - 두 번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달

- randomizedSearchCV : 교차 검증으로 핸덤한 하이퍼파라미터 탐색 수행
  - 최상의 모델 찾은 후 훈련세트 전체를 사용해 최종모델 훈련
  - 첫번째 매개변수로 그리드 서치 수행할 모델 객체 전달
  - 두번째 매개변수로 탐색할 매개변수와 확률 분포 객체 전달

=> scoring, cv, n_jobs, return_train_score 매개변수 -> cross _validate()와 동일
